In [ ]:
#Install
!pip install requests pandas numpy tqdm

Defaulting to user installation because normal site-packages is not writeable


In [1]:
#Import
import requests
import pandas as pd
import numpy as np
import http.client, json
from urllib.parse import urlparse
import json
from tqdm import tqdm

## Premières requêtes

In [2]:
def get_data(url):
    parsed_url = urlparse(url)
    conn = http.client.HTTPConnection(parsed_url.netloc)
    conn.request("GET", parsed_url.path)
    res = conn.getresponse()
    data = res.read()
    jsondata = json.loads(data.decode("utf-8"))
    return jsondata

In [3]:
def get_match_data(match_data):
    d ={}

    d['homeTeam_name'] = match_data['event']['homeTeam']['name']
    d['homeTeam_id'] = match_data['event']['homeTeam']['id']
    d['homeTeam_score'] = match_data['event']['homeScore']['normaltime']

    d['awayTeam_name'] = match_data['event']['awayTeam']['name']
    d['awayTeam_id'] = match_data['event']['awayTeam']['id']
    d['awayTeam_score'] = match_data['event']['awayScore']['normaltime']
    d['date'] = match_data['event']['startTimestamp']

    return d

idée maintenant : créer un dictionnaire : 
- key=joueur(id), 
- value = dict= (key=matchs joués par le joueur, value=statistics)

https://www.sofascore.com/api/v1/unique-tournament/17/season/52186/events/last/0
-> donne les id des matchs du championnat 17 sur la saisons 52186

https://www.sofascore.com/api/v1/event/12436466/lineups
-> donne les stats par joueur du match avec l'id 12436466

## Exemple sur la saison 23-24
1) On recupère la liste des id des matchs de la saison
2) on boucle sur les matchs
    -> sur chaque match, on ajoute a notre dictionnaire : 
        -> joueur : stat du match

In [4]:
players_data = {}
matches_id = []

In [5]:
url_root = 'https://www.sofascore.com/api/v1/unique-tournament/17/season/52186/events/last/'
hasNextPage = True
i = 0

while hasNextPage:
    present_url = url_root + str(i)
    data = get_data(present_url)
    for match in data['events']:
        matches_id.append(match['id'])
    hasNextPage = data['hasNextPage'] #True or False
    i+=1 #access to nextpage

In [6]:
len(matches_id)

380

Il y a 38 journées de championnat et 10 match par journée donc 380 est le bon résultat

In [7]:
example_id = matches_id[0]
example_id

11352485

In [8]:
match_data = get_data('https://www.sofascore.com/api/v1/event/11352485')

In [9]:
for k,v in match_data['event'].items():
    print(k)
    print(v)
    print('\n')

tournament
{'name': 'Premier League', 'slug': 'premier-league', 'category': {'name': 'England', 'slug': 'england', 'sport': {'name': 'Football', 'slug': 'football', 'id': 1}, 'id': 1, 'country': {'alpha2': 'EN', 'alpha3': 'ENG', 'name': 'England', 'slug': 'england'}, 'flag': 'england', 'alpha2': 'EN'}, 'uniqueTournament': {'name': 'Premier League', 'slug': 'premier-league', 'primaryColorHex': '#3c1c5a', 'secondaryColorHex': '#f80158', 'category': {'name': 'England', 'slug': 'england', 'sport': {'name': 'Football', 'slug': 'football', 'id': 1}, 'id': 1, 'country': {'alpha2': 'EN', 'alpha3': 'ENG', 'name': 'England', 'slug': 'england'}, 'flag': 'england', 'alpha2': 'EN'}, 'userCount': 1378471, 'id': 17, 'country': {}, 'hasPerformanceGraphFeature': True, 'hasEventPlayerStatistics': True, 'displayInverseHomeAwayTeams': False, 'fieldTranslations': {'nameTranslation': {'ar': 'الدوري الإنجليزي الممتاز', 'hi': 'प्रिमियर लीग'}, 'shortNameTranslation': {}}}, 'priority': 617, 'isGroup': False, 'c

In [34]:
dataset_dict = {}

for id in tqdm(matches_id):
    match_id = str(id)
    try:

        data = get_data(f'https://www.sofascore.com/api/v1/event/{id}/lineups')
        match_json = get_data(f"https://www.sofascore.com/api/v1/event/{match_id}")
        event_data = get_match_data(match_json)
        if data['confirmed']:
            for team in ["home","away"]:
                for player in data[team]['players']:
                    player_id = str(player['player']['id'])

                    if player_id not in dataset_dict: #si Joueur n'est pas dans le dataset

                        player_info = { 'name': player['player']['name'], 'position': player['player']['position'], 'id': player['player']['id']}

                        dataset_dict[player_id] = {'player_info': player_info, 'matches': {}}  # on lui crée un dictionnaire
                        
                    if match_id not in dataset_dict[player_id]['matches']:
                        dataset_dict[player_id]['matches'][match_id] = {}

                    dataset_dict[player_id]['matches'][match_id]['player_statistics'] = player['statistics']
                    dataset_dict[player_id]['matches'][match_id]['match_info'] = match_data
                    

    except Exception as e:
        print(f"Erreur lors du traitement du match {match_id}: {e}")

        continue

100%|██████████| 380/380 [00:15<00:00, 24.73it/s]


{
    "player_id": {
        "info": {
            ...  // Informations générales du joueur
        },
        "matches": {
            "match_id": {
                "statistics": {
                    ...  // Statistiques du joueur pour ce match
                },
                "info": {
                    ...  // Informations générales du match
                }
            }
        }
    }
}

In [11]:
dataset_dict.keys()

dict_keys(['171919', '576384', '958916', '855864', '858195', '907072', '135256', '814873', '927083', '290927', '972421', '826057', '1031258', '805123', '885864', '1006387', '1005959', '26404', '548050', '268903', '103335', '329417', '825719', '861972', '803174', '364612', '371222', '352770', '285949', '31417', '1015256', '286165', '192442', '82566', '151499', '355528', '358694', '190877', '118179', '996985', '888971', '1009334', '856719', '1117223', '914477', '792122', '793167', '1142679', '358394', '809276', '901907', '875360', '49805', '1090458', '990550', '980643', '166381', '552632', '826183', '26989', '42209', '980634', '150396', '99090', '1136730', '372336', '866191', '866469', '914902', '113956', '823941', '994546', '855647', '560128', '333373', '42699', '162653', '1125214', '185995', '994550', '254491', '44614', '383560', '149663', '964994', '827606', '136710', '331209', '70996', '859765', '839956', '152077', '945122', '189061', '934386', '944656', '125274', '1136731', '318941'

In [12]:
dataset_dict["171919"].keys()

dict_keys(['player_info', 'matches'])

In [35]:
# Display player information
dataset_dict["171919"]['player_info']

{'name': 'Mark Flekken', 'position': 'G', 'id': 171919}

In [14]:
dataset_dict["171919"]['matches'].keys()

dict_keys(['11352485', '11352519', '11352549', '11352434', '11352457', '11352614', '11352325', '11352355', '11352394', '11352530', '11352572', '11352599', '11352446', '11352450', '11971437', '11352521', '11352313', '11352351', '11352380', '11352547', '11352576', '11352421', '11352471', '11352487', '11352339', '11352356', '11352393', '11352660', '11352672', '11352304', '11352628', '11352643', '11352456', '11352543', '11352615', '11352309', '11352358'])

In [15]:
dataset_dict["171919"]['matches']['11352485'].keys()

dict_keys(['player_statistics', 'match_info'])

In [142]:
# Écrire le JSON dans un fichier
import json

with open('dataset.json', 'w') as json_file:
    json.dump(dataset_dict, json_file, indent=4)


## Modélisation

In [25]:
player_ids = dataset_dict.keys()
list(player_ids)

['171919',
 '576384',
 '958916',
 '855864',
 '858195',
 '907072',
 '135256',
 '814873',
 '927083',
 '290927',
 '972421',
 '826057',
 '1031258',
 '805123',
 '885864',
 '1006387',
 '1005959',
 '26404',
 '548050',
 '268903',
 '103335',
 '329417',
 '825719',
 '861972',
 '803174',
 '364612',
 '371222',
 '352770',
 '285949',
 '31417',
 '1015256',
 '286165',
 '192442',
 '82566',
 '151499',
 '355528',
 '358694',
 '190877',
 '118179',
 '996985',
 '888971',
 '1009334',
 '856719',
 '1117223',
 '914477',
 '792122',
 '793167',
 '1142679',
 '358394',
 '809276',
 '901907',
 '875360',
 '49805',
 '1090458',
 '990550',
 '980643',
 '166381',
 '552632',
 '826183',
 '26989',
 '42209',
 '980634',
 '150396',
 '99090',
 '1136730',
 '372336',
 '866191',
 '866469',
 '914902',
 '113956',
 '823941',
 '994546',
 '855647',
 '560128',
 '333373',
 '42699',
 '162653',
 '1125214',
 '185995',
 '994550',
 '254491',
 '44614',
 '383560',
 '149663',
 '964994',
 '827606',
 '136710',
 '331209',
 '70996',
 '859765',
 '839956',

In [30]:
player1_id = str(list(player_ids)[0])
dataset_dict[player1_id].keys()

dict_keys(['player_info', 'matches'])

In [36]:
player1 = dataset_dict[player1_id]
player1['player_info']

{'name': 'Mark Flekken', 'position': 'G', 'id': 171919}

name
position
id

In [43]:
match_id = list(player1['matches'].keys())[0]
match = player1['matches'][match_id]
match.keys()

dict_keys(['player_statistics', 'match_info'])

In [44]:
match['player_statistics']

{'totalPass': 43,
 'accuratePass': 38,
 'totalLongBalls': 16,
 'accurateLongBalls': 11,
 'goodHighClaim': 4,
 'savedShotsFromInsideTheBox': 3,
 'saves': 3,
 'minutesPlayed': 90,
 'touches': 58,
 'rating': 8,
 'possessionLostCtrl': 5,
 'ratingVersions': {'original': 8, 'alternative': 7.7},
 'goalsPrevented': 0.24}

In [50]:
for k,v in match['match_info']['event'].items():
    print(k)
    print(v)
    print('\n')

tournament
{'name': 'Premier League', 'slug': 'premier-league', 'category': {'name': 'England', 'slug': 'england', 'sport': {'name': 'Football', 'slug': 'football', 'id': 1}, 'id': 1, 'country': {'alpha2': 'EN', 'alpha3': 'ENG', 'name': 'England', 'slug': 'england'}, 'flag': 'england', 'alpha2': 'EN'}, 'uniqueTournament': {'name': 'Premier League', 'slug': 'premier-league', 'primaryColorHex': '#3c1c5a', 'secondaryColorHex': '#f80158', 'category': {'name': 'England', 'slug': 'england', 'sport': {'name': 'Football', 'slug': 'football', 'id': 1}, 'id': 1, 'country': {'alpha2': 'EN', 'alpha3': 'ENG', 'name': 'England', 'slug': 'england'}, 'flag': 'england', 'alpha2': 'EN'}, 'userCount': 1378471, 'id': 17, 'country': {}, 'hasPerformanceGraphFeature': True, 'hasEventPlayerStatistics': True, 'displayInverseHomeAwayTeams': False, 'fieldTranslations': {'nameTranslation': {'ar': 'الدوري الإنجليزي الممتاز', 'hi': 'प्रिमियर लीग'}, 'shortNameTranslation': {}}}, 'priority': 617, 'isGroup': False, 'c

In [49]:
match['match_info']['event']

{'tournament': {'name': 'Premier League',
  'slug': 'premier-league',
  'category': {'name': 'England',
   'slug': 'england',
   'sport': {'name': 'Football', 'slug': 'football', 'id': 1},
   'id': 1,
   'country': {'alpha2': 'EN',
    'alpha3': 'ENG',
    'name': 'England',
    'slug': 'england'},
   'flag': 'england',
   'alpha2': 'EN'},
  'uniqueTournament': {'name': 'Premier League',
   'slug': 'premier-league',
   'primaryColorHex': '#3c1c5a',
   'secondaryColorHex': '#f80158',
   'category': {'name': 'England',
    'slug': 'england',
    'sport': {'name': 'Football', 'slug': 'football', 'id': 1},
    'id': 1,
    'country': {'alpha2': 'EN',
     'alpha3': 'ENG',
     'name': 'England',
     'slug': 'england'},
    'flag': 'england',
    'alpha2': 'EN'},
   'userCount': 1378471,
   'id': 17,
   'country': {},
   'hasPerformanceGraphFeature': True,
   'hasEventPlayerStatistics': True,
   'displayInverseHomeAwayTeams': False,
   'fieldTranslations': {'nameTranslation': {'ar': 'الدور

tournament['id']
season['id']
'winnerCode'
homeTeam[id, name]
awayTeam[id,name]
homescore['normaltime]
awayscore['normaltime]
starttimeStamp

In [53]:
match = match['match_info']['event']

In [57]:
match['winnerCode']

3

In [ ]:
d = { 'season_id' : match['season']['id'], 'winnerCode' : match['winnerCode'], 'homeTeam_id' : match['homeTeam']['id'],  }